In [1]:
from langchain_cohere import ChatCohere

llm = ChatCohere()

In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [3]:
from langchain_cohere.embeddings import CohereEmbeddings

embeddings = CohereEmbeddings()

In [4]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [5]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [6]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by visualizing test results.'

In [7]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [8]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith is a platform that supports LLM application development, monitoring, and testing. It offers a range of features to facilitate testing and improve the performance of LLM applications. 

- Initial Test Set: LangSmith enables developers to create datasets containing inputs and reference outputs to run tests on their LLM applications. These test cases can be created, uploaded in bulk, or exported from application traces. 
- Comparison View: LangSmith allows users to compare the performance of different application versions side by side. This helps identify regressions and improvements across multiple revisions of the application. 
- Playground: The playground environment offers a space for rapid experimentation and testing of different prompts and models. Each playground run is logged and can be used to create test cases or for comparison. 
- Beta Testing: During beta testing, LangSmith helps collect data on the LLM application's performance in real-world scenarios. It captures u